In [1]:
import pandas as pd

# 1. Inladen
excel_file = 'studentenprognoses-2025.xlsx'
excel_sheet = 'i v o s t h'
csv_file = '04.-inschrijvingen-wo-2024.csv'

# 2. Kolomnamen opschonen
def clean_cols(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_', regex=False)

# Prognoses
df_prog = pd.read_excel(excel_file, sheet_name=excel_sheet)
clean_cols(df_prog)

# Alleen relevante kolommen
prognose_cols = [
    'instellingscode', 'instellingsnaam', 'jaar',
    'instroom_ho', 'tweedejaars_ho', 'hogerejaars_ho'
]
df_prog = df_prog[prognose_cols]
df_prog['jaar'] = df_prog['jaar'].astype(int)

# Inschrijvingen
df_ins = pd.read_csv(csv_file, sep=';')
clean_cols(df_ins)

# Inschrijvingen naar long-form
jaar_cols = [col for col in df_ins.columns if col.isdigit()]
id_cols = [col for col in df_ins.columns if col not in jaar_cols]
df_ins_long = df_ins.melt(id_vars=id_cols, value_vars=jaar_cols,
                          var_name='jaar', value_name='inschrijvingen')

# Waarden opschonen en types zetten
df_ins_long['jaar'] = df_ins_long['jaar'].astype(int)
df_ins_long['inschrijvingen'] = pd.to_numeric(
    df_ins_long['inschrijvingen'].astype(str).str.extract(r'(\d+)', expand=False),
    errors='coerce'
).fillna(0).astype(int)

# Kolomnaam harmoniseren
if 'instellingscode_actueel' in df_ins_long.columns:
    df_ins_long.rename(columns={'instellingscode_actueel': 'instellingscode'}, inplace=True)

# Houd alleen nodige kolommen
df_ins_clean = df_ins_long[['instellingscode', 'jaar', 'instellingsnaam_actueel', 'inschrijvingen']]
df_ins_clean.rename(columns={'instellingsnaam_actueel': 'instellingsnaam'}, inplace=True)

# 🧠 Merge met prognoses (LEFT JOIN zodat toekomstige jaren blijven)
df_merge = pd.merge(df_prog, df_ins_clean, on=['instellingscode', 'jaar'], how='left')

# Vul eventueel ontbrekende inschrijvingen met 0
df_merge['inschrijvingen'] = df_merge['inschrijvingen'].fillna(0).astype(int)

# Prognosekolommen hernoemen
df_merge.rename(columns={
    'instroom_ho': 'prognose_instroom',
    'tweedejaars_ho': 'prognose_tweedejaars',
    'hogerejaars_ho': 'prognose_hogerejaars'
}, inplace=True)

# Sorteren
df_merge.sort_values(['instellingscode', 'jaar'], inplace=True)

# Opslaan
df_merge.to_csv('studentdata_preprocessed.csv', index=False)
print(f"✅ Gereed: {df_merge.shape[0]} rijen opgeslagen, inclusief toekomstige jaren.")


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.